In [1]:
import xml.etree.ElementTree as ET
import math
ET.register_namespace('android', 'http://schemas.android.com/apk/res/android')

In [2]:
erros = []
componenteErros = []
componenteIdErros = []

In [3]:
#Guideline 1.1 Text Alternatives
#1.1.1 Non-text Content
#Level A
#Não foi tratado label para cada input, pois não é necessário em aplicativos móveis

def criterio111(element):
    
    #Imagens e botões com imagens com textos alternativos
    #Imagens e botões com imagens com null em contentDescription
    if element.tag == 'ImageView' or element.tag == 'ImageButton':
        if '{http://schemas.android.com/apk/res/android}contentDescription' in element.attrib:
            value = element.attrib['{http://schemas.android.com/apk/res/android}contentDescription']
            if(value == '@null'):
                componenteErros.append(ET.tostring(element, encoding='utf8').decode('utf8'))
                if '{http://schemas.android.com/apk/res/android}id' in element.attrib:
                    componenteIdErros.append(element.attrib['{http://schemas.android.com/apk/res/android}id'])
                erros.append('1.1.1 - Conteúdo não textual')
        else:
            componenteErros.append(ET.tostring(element, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in element.attrib:
                componenteIdErros.append(element.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.1.1 - Conteúdo não textual')
            
    #Inputs de texto em formulários possuem texto descritivo
    #Inputs de texto em formulários possuem texto descritivo não nulo
    if element.tag == 'EditText':
        if '{http://schemas.android.com/apk/res/android}hint' in element.attrib:
            value = element.attrib['{http://schemas.android.com/apk/res/android}hint']
            if(value == ''):
                componenteErros.append(ET.tostring(element, encoding='utf8').decode('utf8'))
                if '{http://schemas.android.com/apk/res/android}id' in element.attrib:
                    componenteIdErros.append(element.attrib['{http://schemas.android.com/apk/res/android}id'])
                erros.append('1.1.1 - Conteúdo não textual')
        else:
            componenteErros.append(ET.tostring(element, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in element.attrib:
                componenteIdErros.append(element.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.1.1 - Conteúdo não textual')
            
    #Botões em formulários possuem texto descritivo
    #Botões em formulários possuem texto descritivo não nulo
    if element.tag == 'Button' or element.tag == 'RadioButton' or element.tag == 'ToggleButton' or element.tag == 'FloatingActionButton':
        if '{http://schemas.android.com/apk/res/android}text' in element.attrib:
            value = element.attrib['{http://schemas.android.com/apk/res/android}text']
            if(value == ''):
                componenteErros.append(ET.tostring(element, encoding='utf8').decode('utf8'))
                if '{http://schemas.android.com/apk/res/android}id' in element.attrib:
                    componenteIdErros.append(element.attrib['{http://schemas.android.com/apk/res/android}id'])
                erros.append('1.1.1 - Conteúdo não textual')
        else:
            componenteErros.append(ET.tostring(element, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in element.attrib:
                componenteIdErros.append(element.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.1.1 - Conteúdo não textual')

In [4]:
#Guideline 1.4 Distinguível
#1.4.3 Contraste Mínimo
#Level AA

def calc_luminanace(v):
    v /= 255
    return  (v / 12.92) if (v <= 0.03928) else (pow( (v + 0.055) / 1.055, 2.4 ));
                
def luminanace(r, g, b):
    r = calc_luminanace(r)
    g = calc_luminanace(g)
    b = calc_luminanace(b)
    return r * 0.2126 + g * 0.7152 + b * 0.0722

def contrast(rgb1, rgb2):
    lum1 = luminanace(rgb1[0], rgb1[1], rgb1[2])
    lum2 = luminanace(rgb2[0], rgb2[1], rgb2[2])
    brightest = max(lum1, lum2)
    darkest = min(lum1, lum2)
    return (brightest + 0.05) / (darkest + 0.05)

def getArrayRGB(hexColor):
    rgb = tuple(int(hexColor[i:i+2], 16) for i in (0, 2, 4))
    array = []
    array.append(rgb)
    array = [x for xs in array for x in xs]
    return array

def criterio143(child, parent):
    if '{http://schemas.android.com/apk/res/android}background' in child.attrib and '{http://schemas.android.com/apk/res/android}textColor' in child.attrib:
        background = child.attrib['{http://schemas.android.com/apk/res/android}background'].lstrip('#')
        textColor = child.attrib['{http://schemas.android.com/apk/res/android}textColor'].lstrip('#')
        arrayB = getArrayRGB(background)
        arrayT = getArrayRGB(textColor)
        if contrast(arrayB, arrayT) < 4.5:
            componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.4.3 - Contraste Mínimo')
    elif '{http://schemas.android.com/apk/res/android}background' in parent.attrib and '{http://schemas.android.com/apk/res/android}textColor' in child.attrib:
        background = parent.attrib['{http://schemas.android.com/apk/res/android}background'].lstrip('#')
        textColor = child.attrib['{http://schemas.android.com/apk/res/android}textColor'].lstrip('#')
        arrayB = getArrayRGB(background)
        arrayT = getArrayRGB(textColor)
        if contrast(arrayB, arrayT) < 4.5:
            componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.4.3 - Contraste Mínimo')

SyntaxError: unexpected EOF while parsing (<ipython-input-5-b69e30b76cfc>, line 1)

In [7]:
tree = ET.parse('activity_new_patient.xml')
root = tree.getroot()
erros = []
componenteErros = []
componenteIdErros = []
parent_map = {c:p for p in tree.iter() for c in p}
for c, p in parent_map.items():
    criterio111(c)
    criterio143(c, p)
print(erros)
print(componenteErros)
print(componenteIdErros)

['1.1.1 - Conteúdo não textual']
['<?xml version=\'1.0\' encoding=\'utf8\'?>\n<ImageView xmlns:android="http://schemas.android.com/apk/res/android" android:contentDescription="@null" android:layout_gravity="center_horizontal|top" android:layout_height="wrap_content" android:layout_marginBottom="-15dp" android:layout_width="75dp" android:src="@drawable/ic_camera_account_white_48dp" />\n\n                ']
[]


In [33]:
[elem.tag for elem in root.iter()]

['RelativeLayout',
 'ScrollView',
 'LinearLayout',
 'LinearLayout',
 'ImageView',
 'android.support.design.widget.TextInputLayout',
 'EditText',
 'android.support.design.widget.TextInputLayout',
 'EditText',
 'android.support.design.widget.TextInputLayout',
 'EditText',
 'android.support.design.widget.TextInputLayout',
 'EditText',
 'android.support.design.widget.TextInputLayout',
 'EditText',
 'RadioGroup',
 'RadioButton',
 'RadioButton',
 'android.support.design.widget.TextInputLayout',
 'EditText',
 'android.support.design.widget.TextInputLayout',
 'EditText',
 'Button']

In [ ]:
print(ET.tostring(root, encoding='utf8').decode('utf8'))

In [22]:
from lxml import etree
from io import StringIO, BytesIO
tree = etree.parse("activity_new_patient.xml")
text = etree.tostring(tree.getroot())
f = StringIO(text)
context = etree.iterparse(f, events=("start", "end"), tag="element")

for action, elem in context:
     print("%s: %s" % (action, elem.tag))

TypeError: initial_value must be str or None, not bytes

In [ ]:
def criterio111(root):
    
    #Imagens e botões com imagens com textos alternativos
    #Imagens e botões com imagens com null em contentDescription
    for child in root.iter('ImageView') or root.iter('ImageButton'):
        if '{http://schemas.android.com/apk/res/android}contentDescription' in child.attrib:
            value = child.attrib['{http://schemas.android.com/apk/res/android}contentDescription']
            if(value == '@null'):
                componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
                if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                    componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
                erros.append('1.1.1 Non-text Content')
        else:
            componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.1.1 Non-text Content')
            
    #Inputs de texto em formulários possuem texto descritivo
    #Inputs de texto em formulários possuem texto descritivo não nulo
    for child in root.iter('EditText'):
        if '{http://schemas.android.com/apk/res/android}hint' in child.attrib:
            value = child.attrib['{http://schemas.android.com/apk/res/android}hint']
            if(value == ''):
                componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
                if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                    componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
                erros.append('1.1.1 Non-text Content')
        else:
            componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.1.1 Non-text Content')
            
    #Botões em formulários possuem texto descritivo
    #Botões em formulários possuem texto descritivo não nulo
    for child in root.iter('Button') or root.iter('RadioButton') or root.iter('ToggleButton') or root.iter('FloatingActionButton'):
        if '{http://schemas.android.com/apk/res/android}text' in child.attrib:
            value = child.attrib['{http://schemas.android.com/apk/res/android}text']
            if(value == ''):
                componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
                if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                    componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
                erros.append('1.1.1 Non-text Content')
        else:
            componenteErros.append(ET.tostring(child, encoding='utf8').decode('utf8'))
            if '{http://schemas.android.com/apk/res/android}id' in child.attrib:
                componenteIdErros.append(child.attrib['{http://schemas.android.com/apk/res/android}id'])
            erros.append('1.1.1 Non-text Content')

In [13]:
h = input('Enter hex: ').lstrip('#')
print('RGB =', tuple(int(h[i:i+2], 16) for i in (0, 2, 4)))

Enter hex: #A8A8A8
RGB = (168, 168, 168)
